In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [2]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
# ## Plotly
# import plotly.offline as py
# import plotly.graph_objs as go
# py.init_notebook_mode(connected=True)
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

from sklearn.manifold import TSNE

Using TensorFlow backend.


In [3]:
ids = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/id.Dennis+Schwartz.txt", sep="[\r\n]+", header=None, names=["id"])
label_3class = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/label.3class.Dennis+Schwartz.txt", sep="[\r\n]+", names=["3class_label"])
label_4class = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/label.4class.Dennis+Schwartz.txt", sep="[\r\n]+",names=["4class_label"])
rating = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/rating.Dennis+Schwartz.txt", sep="[\r\n]+",names=["rating"])
subj = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/subj.Dennis+Schwartz.txt", sep="[\r\n]+",names=["subj_extraction"])

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support 

In [4]:
dennis = pd.concat([ids,label_3class, label_4class, rating, subj], axis = 1)
dennis.head(5)

,id,3class_label,4class_label,rating,subj_extraction
0,29420,0,0,0.1,"in my opinion , a movie reviewer's most import..."
1,17219,0,0,0.2,"you can watch this movie , that is based on a ..."
2,18406,0,0,0.2,"this is asking a lot to believe , and though i..."
3,18648,0,0,0.2,no heroes and no story are the main attributes...
4,20021,0,0,0.2,"this is not an art movie , yet i saw it an art..."


In [5]:
dennis["strongly neg"]=dennis.apply(lambda x:x["4class_label"]==0, axis=1)
dennis["neg"]=dennis.apply(lambda x:x["4class_label"]==1, axis=1)
dennis["pos"]=dennis.apply(lambda x:x["4class_label"]==2, axis=1)
dennis["strongly pos"]=dennis.apply(lambda x:x["4class_label"]==3, axis=1)

In [32]:
y_d = dennis['3class_label']
dennis['n']=dennis.apply(lambda x:x["4class_label"]==0, axis=1)
dennis["neg"]=dennis.apply(lambda x:x["4class_label"]==1, axis=1)
dennis["pos"]=dennis.apply(lambda x:x["4class_label"]==2, axis=1)
y_d_encode = dennis[['n', 'neg', 'pos']]
x_d = list(dennis['subj_extraction'])

In [7]:
from sklearn.model_selection import train_test_split
xtrain_d, xvalid_d, ytrain_d, yvalid_d = train_test_split(list(dennis['subj_extraction']), y_d, 
                                                  stratify=y_d, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [8]:
xtrain_d, xvalid_d, ytrain_d_encode, yvalid_d_encode = train_test_split(dennis['subj_extraction'], y_d_encode, 
                                                  stratify=y_d, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [49]:
import glove_dl22 as gd

In [ ]:
# use total_text to create embedding matrix
total_text = x_d + x_j

In [50]:
vocab_size, embedding_matrix, padded_docs = gd.get_glove_m(total_text)

In [51]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
         0.82779998,  0.27061999],
       [-0.1529    , -0.24279   ,  0.89837003, ..., -0.59100002,
         1.00390005,  0.20664001],
       ..., 
       [ 0.19193999,  0.38505   , -0.22404   , ..., -0.16169   ,
         0.080152  ,  0.29284999],
       [-0.045712  ,  0.14199001, -0.30963001, ..., -0.22722   ,
         0.76585001,  0.66973001],
       [ 0.0068313 , -0.15109   , -0.012474  , ..., -0.019687  ,
         0.33684   ,  0.41115999]])

In [63]:
# Covert xtest and xtrain to padded doc
xvalid_d_padded_docs = gd.padded_doc(xvalid_d)
xtrain_d_padded_docs = gd.padded_doc(xtrain_d)

## cnn 

In [80]:

from keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Convolution1D, Embedding
from keras.layers.merge import Concatenate
from keras.models import Sequential, Model
def build_model(embedding_matrix, vocab_size):
    # define model
    embedding_dim = 50
    filter_sizes = (2, 4)
    num_filters = 10
    dropout_prob = (0.5, 0.8)
    hidden_dims = 50

    # Build model
    sequence_length = 100
    input_shape = (sequence_length,)

    model_input = Input(shape=input_shape)

    # Static model does not have embedding layer

    z = Embedding(vocab_size, 100, input_length=sequence_length,weights=[embedding_matrix], name="embedding")(model_input)
    z.trainable = False
    
    z = Dropout(dropout_prob[0])(z)

    # Convolutional block
    conv_blocks = []
    for sz in filter_sizes:
        conv = Convolution1D(filters=num_filters,
                             kernel_size=sz,
                             padding="valid",
                             activation="relu",
                             strides=1)(z)
        conv = MaxPooling1D(pool_size=1)(conv)
        conv = Flatten()(conv)
        conv_blocks.append(conv)
    z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]

    z = Dropout(dropout_prob[1])(z)
    z = Dense(hidden_dims, activation="relu")(z)
    model_output = Dense(3, activation="sigmoid")(z)

    model = Model(model_input, model_output)
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    weights = np.array([v for v in embedding_matrix])
    print("Initializing embedding layer with glove weights, shape", weights.shape)
    embedding_layer = model.get_layer("embedding")
    embedding_layer.set_weights([weights])
    return model

# # Initialize weights with word2vec
# if model_type == "CNN-non-static":
#     weights = np.array([v for v in embedding_weights.values()])
#     print("Initializing embedding layer with word2vec weights, shape", weights.shape)
#     embedding_layer = model.get_layer("embedding")
#     embedding_layer.set_weights([weights])

# Train the model
model = build_model(embedding_matrix, vocab_size)
model.fit(xtrain_d_padded_docs, ytrain_d_encode, epochs=10, verbose=1)

Initializing embedding layer with glove weights, shape (33501, 100)
Epoch 1/10
924/924 [==============================] - 4s 4ms/step - loss: 0.9929 - acc: 0.4004
Epoch 2/10
924/924 [==============================] - 3s 3ms/step - loss: 0.9512 - acc: 0.4102
Epoch 3/10
924/924 [==============================] - 3s 3ms/step - loss: 0.9446 - acc: 0.4242
Epoch 4/10
924/924 [==============================] - 3s 3ms/step - loss: 0.9397 - acc: 0.4123
Epoch 5/10
924/924 [==============================] - 3s 3ms/step - loss: 0.9444 - acc: 0.4361
Epoch 6/10
924/924 [==============================] - 3s 3ms/step - loss: 0.9228 - acc: 0.4167
Epoch 7/10
924/924 [==============================] - 3s 3ms/step - loss: 0.9166 - acc: 0.4264
Epoch 8/10
924/924 [==============================] - 3s 3ms/step - loss: 0.9264 - acc: 0.4307
Epoch 9/10
924/924 [==============================] - 3s 3ms/step - loss: 0.9052 - acc: 0.4372
Epoch 10/10
924/924 [==============================] - 3s 3ms/step - loss: 0.

In [23]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 400, 100)     1791100     input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 400, 100)     0           embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 399, 10)      2010        dropout_1[0][0]                  
__________________________________________________________________________________________________
conv1d_2 (

## visuliaze the prediction

In [75]:
# evaluate the model
loss, accuracy = model.evaluate(xvalid_d_padded_docs, yvalid_d_encode, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 41.747573


In [27]:
xtrain_d_glv = np.array(xtrain_d_glv)
xvalid_d_glv = np.array(xvalid_d_glv)
xtrain_d_glv.shape

(924, 300)

In [18]:
# serialize model to JSON
model_json = model.to_json()
with open("model_gl.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_gl.h5")
print("Saved model to disk")

Saved model to disk


In [81]:
# serialize model to JSON
model_cnn_json = model.to_json()
with open("model_gl_cnn.json", "w") as json_file:
    json_file.write(model_cnn_json)
# serialize weights to HDF5
model.save_weights("model_gl_cnn.h5")
print("Saved model to disk")

Saved model to disk


# glove_nn on the other writer

In [20]:
ids = pd.read_csv("scale_data/scaledata/James+Berardinelli/id.James+Berardinelli.txt", sep="[\r\n]+", header=None, names=["id"])
# label_3class = pd.read_csv("scale_data/scaledata/James+Berardinelli/label.3class.James+Berardinelli.txt", sep="[\r\n]+", names=["3class_label"])
label_3class = pd.read_csv("scale_data/scaledata/James+Berardinelli/label.3class.James+Berardinelli.txt", sep="[\r\n]+",names=["3class_label"])

label_4class = pd.read_csv("scale_data/scaledata/James+Berardinelli/label.4class.James+Berardinelli.txt", sep="[\r\n]+",names=["4class_label"])
rating = pd.read_csv("scale_data/scaledata/James+Berardinelli/rating.James+Berardinelli.txt", sep="[\r\n]+",names=["rating"])
subj = pd.read_csv("scale_data/scaledata/James+Berardinelli/subj.James+Berardinelli.txt", sep="[\r\n]+",names=["subj_extraction"])

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: ParserWarnin

In [21]:
james = pd.concat([ids,label_3class, label_4class, rating, subj], axis = 1)


In [22]:
james["strongly neg"]=james.apply(lambda x:x["4class_label"]==0, axis=1)
james["neg"]=james.apply(lambda x:x["4class_label"]==1, axis=1)
james["pos"]=james.apply(lambda x:x["4class_label"]==2, axis=1)
james["strongly pos"]=james.apply(lambda x:x["4class_label"]==3, axis=1)

In [23]:
y_d = dennis['3class_label']
james['n']=james.apply(lambda x:x["4class_label"]==0, axis=1)
james["neg"]=james.apply(lambda x:x["4class_label"]==1, axis=1)
james["pos"]=james.apply(lambda x:x["4class_label"]==2, axis=1)
y_j_encode = james[['n', 'neg', 'pos']]

In [24]:
x_j = james.subj_extraction.values
y_j = james['3class_label']


In [25]:
from sklearn.model_selection import train_test_split
xtrain_j, xvalid_j, ytrain_j, yvalid_j = train_test_split(x_j, y_j, 
                                                  stratify=y_j, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [26]:
xtrain_j, xvalid_j, ytrain_j_encode, yvalid_j_encode = train_test_split(x_j, y_j_encode, 
                                                  stratify=y_j, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [60]:
xtrain_j_glv = [g.sentence_vectorize(sent) for sent in xtrain_j]
xvalid_j_glv = [g.sentence_vectorize(sent) for sent in xvalid_j]
xtrain_j_glv = np.array(xtrain_j_glv)
xvalid_j_glv = np.array(xvalid_j_glv)

xtrain_j_glv.shape

(1176, 300)

In [27]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy
import os

In [52]:
xvalid_j_padded = gd.padded_doc(xvalid_j)
xvalid_j_padded.shape
xtrain_j_padded = gd.padded_doc(xtrain_j)
xtrain_j_padded.shape
# xtrain_j_padded = xtrain_j_padded[:924,:]
# yvalid_j_encode.shape

(1176, 100)

## load the pre_trained model and evaluate on james directly

In [82]:

# load json and create model
json_file = open('model_gl_cnn.json', 'r')
loaded_model_cnn_json = json_file.read()
json_file.close()
loaded_model_cnn = model_from_json(loaded_model_cnn_json)
# load weights into new model
loaded_model_cnn.load_weights("model_gl_cnn.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model_cnn.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model_cnn.evaluate(xvalid_j_padded, yvalid_j_encode, verbose=1)
print("%s: %.2f%%" % (loaded_model_cnn.metrics_names[1], score[1]*100))
# loaded_model.summary()


Loaded model from disk
131/131 [==============================] - 1s 4ms/step
acc: 39.44%


In [57]:
total_text = x_d+x_j 

In [58]:
vocab_size, embedding_matrix, padded_docs = gd.get_glove_m(total_text)


In [59]:
embedding_matrix.shape

(33501, 100)

In [ ]:
model.load

## fine tuning freeze the E layer.

In [89]:
for layer in loaded_model_cnn.layers[:2]:
    layer.trainable = False

In [84]:
loaded_model_cnn

In [85]:
from keras import optimizers

loaded_model_cnn.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [86]:
model.fit(xtrain_j_padded, ytrain_j_encode, epochs = 10, verbose = 1)

Epoch 1/10
1176/1176 [==============================] - 4s 4ms/step - loss: 0.8200 - acc: 0.3104
Epoch 2/10
1176/1176 [==============================] - 3s 3ms/step - loss: 0.7752 - acc: 0.4337
Epoch 3/10
1176/1176 [==============================] - 3s 3ms/step - loss: 0.7654 - acc: 0.4515
Epoch 4/10
1176/1176 [==============================] - 3s 3ms/step - loss: 0.7507 - acc: 0.4541
Epoch 5/10
1176/1176 [==============================] - 4s 3ms/step - loss: 0.7538 - acc: 0.4549
Epoch 6/10
1176/1176 [==============================] - 3s 3ms/step - loss: 0.7358 - acc: 0.4575
Epoch 7/10
1176/1176 [==============================] - 3s 3ms/step - loss: 0.7264 - acc: 0.4549
Epoch 8/10
1176/1176 [==============================] - 4s 3ms/step - loss: 0.7252 - acc: 0.4541A: 0s - loss: 0.7286 - acc:
Epoch 9/10
1176/1176 [==============================] - 3s 3ms/step - loss: 0.7224 - acc: 0.4558
Epoch 10/10
1176/1176 [==============================] - 4s 3ms/step - loss: 0.7030 - acc: 0.4566


In [90]:
loaded_model_cnn.fit(xtrain_j_padded, ytrain_j_encode, epochs=10, verbose=1)

Epoch 1/10
  64/1176 [>.............................] - ETA: 2s - loss: 0.7982 - acc: 0.5104

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/engine/training.py:953: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1176/1176 [==============================] - 2s 2ms/step - loss: 0.7868 - acc: 0.5105
Epoch 2/10
1176/1176 [==============================] - 2s 2ms/step - loss: 0.7680 - acc: 0.5162
Epoch 3/10
1176/1176 [==============================] - 2s 2ms/step - loss: 0.7587 - acc: 0.5156
Epoch 4/10
1176/1176 [==============================] - 2s 2ms/step - loss: 0.7346 - acc: 0.5334
Epoch 5/10
1176/1176 [==============================] - 2s 2ms/step - loss: 0.7385 - acc: 0.5357
Epoch 6/10
1176/1176 [==============================] - 2s 2ms/step - loss: 0.7319 - acc: 0.5266
Epoch 7/10
1176/1176 [==============================] - 2s 2ms/step - loss: 0.7068 - acc: 0.5468
Epoch 8/10
1176/1176 [==============================] - 2s 2ms/step - loss: 0.7124 - acc: 0.5454
Epoch 9/10
1176/1176 [==============================] - 2s 2ms/step - loss: 0.7007 - acc: 0.5496
Epoch 10/10
1176/1176 [==============================] - 2s 2ms/step - loss: 0.6894 - acc: 0.5683


## the third writer

In [91]:
ids = pd.read_csv("scale_data/scaledata/Scott+Renshaw/id.Scott+Renshaw.txt", sep="[\r\n]+", header=None, names=["id"])
label_3class = pd.read_csv("scale_data/scaledata/Scott+Renshaw/label.3class.Scott+Renshaw.txt", sep="[\r\n]+", names=["3class_label"])
label_4class = pd.read_csv("scale_data/scaledata/Scott+Renshaw/label.4class.Scott+Renshaw.txt", sep="[\r\n]+",names=["4class_label"])
rating = pd.read_csv("scale_data/scaledata/Scott+Renshaw/rating.Scott+Renshaw.txt", sep="[\r\n]+",names=["rating"])
subj = pd.read_csv("scale_data/scaledata/Scott+Renshaw/subj.Scott+Renshaw.txt", sep="[\r\n]+",names=["subj_extraction"])

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support 

In [104]:
scott = pd.concat([ids,label_3class, label_4class, rating, subj], axis = 1)
# y_s is a column with 3 values: 0,1,2
y_s = scott['3class_label']
scott['n']=scott.apply(lambda x:x["3class_label"]==0, axis=1)
scott["neg"]=scott.apply(lambda x:x["3class_label"]==1, axis=1)
scott["pos"]=scott.apply(lambda x:x["3class_label"]==2, axis=1)
# y_s_encode are three columns with binary values for each column
y_s_encode = scott[['n', 'neg', 'pos']]
x_s = scott['subj_extraction']


## the fourth writer

In [98]:
ids = pd.read_csv("scale_data/scaledata/Steve+Rhodes/id.Steve+Rhodes.txt", sep="[\r\n]+", header=None, names=["id"])
label_3class = pd.read_csv("scale_data/scaledata/Steve+Rhodes/label.3class.Steve+Rhodes.txt", sep="[\r\n]+", names=["3class_label"])
label_4class = pd.read_csv("scale_data/scaledata/Steve+Rhodes/label.4class.Steve+Rhodes.txt", sep="[\r\n]+",names=["4class_label"])
rating = pd.read_csv("scale_data/scaledata/Steve+Rhodes/rating.Steve+Rhodes.txt", sep="[\r\n]+",names=["rating"])
subj = pd.read_csv("scale_data/scaledata/Steve+Rhodes/subj.Steve+Rhodes.txt", sep="[\r\n]+",names=["subj_extraction"])

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support 

In [105]:
Rhodes = pd.concat([ids,label_3class, label_4class, rating, subj], axis = 1)
# y_s is a column with 3 values: 0,1,2
y_r = scott['3class_label']
Rhodes['n']=Rhodes.apply(lambda x:x["3class_label"]==0, axis=1)
Rhodes["neg"]=Rhodes.apply(lambda x:x["3class_label"]==1, axis=1)
Rhodes["pos"]=Rhodes.apply(lambda x:x["3class_label"]==2, axis=1)
# y_s_encode are three columns with binary values for each column
y_r_encode = Rhodes[['n', 'neg', 'pos']]
x_r = Rhodes['subj_extraction']

In [106]:
# total text
x_d = list(x_d)
x_j = list(x_j)
x_s = list(x_s)
x_r = list(x_r)
total_text = x_d+x_j+x_s+x_r

In [107]:
# get embedding_matrix for total_text
vocab_size, embedding_matrix, padded_docs = gd.get_glove_m(total_text)

## Train the model from the first writer

In [108]:
pretrainModel_cnn = build_model(embedding_matrix, vocab_size)

Initializing embedding layer with glove weights, shape (41441, 100)
